# Custom Chatbot Project

## CERN is celebrating its 70th anniversary 
During a crucial period for high-energy physics, coinciding with the initiation of the third update to the European strategy for particle physics. In this special edition of CERN Courier magazine, early-career researchers share their visions for the future of the field while reflecting on CERN's scientific and societal contributions. The magazine features expert insights into the achievements of the Large Hadron Collider (LHC) and explores the advancements of the hybrid pixel detector technology, emphasizing its applications beyond particle physics.

## The CERN Courier website 
is a rich repository of articles covering a wide array of topics in particle physics, high-energy physics, and associated technological advancements. It provides in-depth reporting on the latest experimental results from CERN and other international laboratories, offering insights into ongoing research and discoveries in the field.

## Last 11 years of the CERN Courier Magazine in PDF
In this dataset I am downloading the Last 11 years of the CERN Courier Magazine.  I will then take this database and then encode it to be used as a Context Window to ask Questions to OpenAI

# 1: Data Wrangling

In [1]:
%%time
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, unquote
import time
from tqdm import tqdm
import re

class CERNPDFCrawler:
    def __init__(self):
        self.base_url = "https://home.cern/resources"
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.download_folder = "cern_pdfs"
        self.processed_article_urls = set()
        self.downloaded_files = set()
        
        if not os.path.exists(self.download_folder):
            os.makedirs(self.download_folder)
        self.load_existing_files()

    def load_existing_files(self):
        for filename in os.listdir(self.download_folder):
            if filename.lower().endswith('.pdf'):
                self.downloaded_files.add(filename)
        print(f"Found {len(self.downloaded_files)} existing PDF files")

    def get_page_content(self, url):
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = self.session.get(url, headers=self.headers)
                response.raise_for_status()
                return response.text
            except requests.RequestException as e:
                if attempt == max_retries - 1:
                    print(f"Error fetching {url}: {e}")
                    return None
                time.sleep(2 ** attempt)
        return None

    def extract_pdf_urls_from_text(self, text):
        """Extract PDF URLs from text content including 'File path:' patterns"""
        pdf_urls = set()
        
        # Look for "File path:" pattern
        file_path_matches = re.finditer(r'File path:\s*(https?://[^\s<>"]+\.pdf)', text, re.IGNORECASE)
        for match in file_path_matches:
            pdf_urls.add(match.group(1))
            
        # Look for direct PDF links
        pdf_link_matches = re.finditer(r'href="(https?://[^\s<>"]+\.pdf)"', text, re.IGNORECASE)
        for match in pdf_link_matches:
            pdf_urls.add(match.group(1))
            
        return pdf_urls

    def find_courier_links(self, page_url):
        content = self.get_page_content(page_url)
        if not content:
            return []
        
        soup = BeautifulSoup(content, 'html.parser')
        courier_links = []
        
        for link in soup.find_all('a', href=True):
            href = link['href']
            if '/resources/courier/' in href or '/record/' in href:
                full_url = urljoin("https://home.cern", href)
                if full_url not in self.processed_article_urls:
                    courier_links.append(full_url)
                    self.processed_article_urls.add(full_url)
        
        return courier_links

    def find_pdf_links(self, article_url):
        """Find all PDF download links on an article page"""
        content = self.get_page_content(article_url)
        if not content:
            return []
        
        pdf_urls = set()
        
        # Extract URLs from text content
        pdf_urls.update(self.extract_pdf_urls_from_text(content))
        
        # Parse with BeautifulSoup for structured extraction
        soup = BeautifulSoup(content, 'html.parser')
        
        # Look for links containing PDF
        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.lower().endswith('.pdf'):
                full_url = urljoin("https://home.cern", href)
                pdf_urls.add(full_url)
        
        return list(pdf_urls)

    def sanitize_filename(self, url):
        """Create a safe filename from URL"""
        filename = unquote(url.split('/')[-1])
        # Remove or replace unsafe characters
        filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
        return filename

    def download_pdf(self, pdf_url, filename):
        if filename in self.downloaded_files:
            print(f"Skipping {filename} - already downloaded")
            return True
            
        try:
            response = self.session.get(pdf_url, headers=self.headers, stream=True)
            response.raise_for_status()
            
            file_path = os.path.join(self.download_folder, filename)
            
            total_size = int(response.headers.get('content-length', 0))
            
            with open(file_path, 'wb') as file, tqdm(
                desc=filename,
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as pbar:
                for data in response.iter_content(chunk_size=1024):
                    size = file.write(data)
                    pbar.update(size)
            
            self.downloaded_files.add(filename)
            return True
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
            return False

    def crawl_and_download(self, start_page=0, end_page=7):
        print(f"Starting CERN PDF crawler (pages {start_page} to {end_page})")
        
        found_pdfs = 0
        downloaded_pdfs = 0
        skipped_pdfs = 0
        failed_downloads = []
        
        for page_num in range(start_page, end_page + 1):
            page_url = f"{self.base_url}?type=52&page={page_num}"
            print(f"\nProcessing page {page_num}...")
            
            courier_links = self.find_courier_links(page_url)
            print(f"Found {len(courier_links)} new article links on page {page_num}")
            
            for article_url in courier_links:
                pdf_urls = self.find_pdf_links(article_url)
                
                for pdf_url in pdf_urls:
                    found_pdfs += 1
                    filename = self.sanitize_filename(pdf_url)
                    
                    print(f"\nFound PDF: {filename}")
                    print(f"URL: {pdf_url}")
                    
                    if filename in self.downloaded_files:
                        print(f"Skipping - already downloaded")
                        skipped_pdfs += 1
                        continue
                        
                    if self.download_pdf(pdf_url, filename):
                        downloaded_pdfs += 1
                    else:
                        failed_downloads.append(filename)
                
                time.sleep(1)
        
        print("\nDownload Summary:")
        print("-" * 20)
        print(f"Total PDFs found: {found_pdfs}")
        print(f"Successfully downloaded: {downloaded_pdfs}")
        print(f"Skipped (already downloaded): {skipped_pdfs}")
        print(f"Failed downloads: {len(failed_downloads)}")
        if failed_downloads:
            print("\nFailed downloads:")
            for fail in failed_downloads:
                print(f"- {fail}")

if __name__ == "__main__":
    crawler = CERNPDFCrawler()
    crawler.crawl_and_download(0, 7)

Found 0 existing PDF files
Starting CERN PDF crawler (pages 0 to 7)

Processing page 0...
Found 15 new article links on page 0

Found PDF: CERNCourier2024MayJun-digitaledition.pdf
URL: https://cds.cern.ch/record/2896932/files/CERNCourier2024MayJun-digitaledition.pdf


CERNCourier2024MayJun-digitaledition.pdf: 100%|████████████████████████████████████████████████████████████████████████████████████████| 14.7M/14.7M [00:01<00:00, 8.40MiB/s]



Found PDF: CERNCourier2024MarApr-digitaledition.pdf
URL: https://cds.cern.ch/record/2893513/files/CERNCourier2024MarApr-digitaledition.pdf


CERNCourier2024MarApr-digitaledition.pdf: 100%|████████████████████████████████████████████████████████████████████████████████████████| 13.5M/13.5M [00:01<00:00, 12.3MiB/s]



Found PDF: CERNCourier2024JanFeb-digitaledition.pdf
URL: https://cds.cern.ch/record/2886335/files/CERNCourier2024JanFeb-digitaledition.pdf


...


Found PDF: CERN Courier May 2016 (Volume 56 Issue 4).pdf
URL: http://cds.cern.ch/record/2146835/files/CERN%20Courier%20May%202016%20(Volume%2056%20Issue%204).pdf


CERN Courier May 2016 (Volume 56 Issue 4).pdf: 100%|████████████████████████████████████████████████████████████████████████████████████| 10.2M/10.2M [00:13<00:00, 801kiB/s]



Found PDF: CERN Courier Mar 2016 (Volume 56 Issue 2).pdf
URL: https://cds.cern.ch/record/2131754/files/CERN%20Courier%20Mar%202016%20(Volume%2056%20Issue%202).pdf


CERN Courier Mar 2016 (Volume 56 Issue 2).pdf: 100%|███████████████████████████████████████████████████████████████████████████████████| 9.46M/9.46M [00:01<00:00, 6.54MiB/s]



Processing page 5...
Found 15 new article links on page 5

Found PDF: CERNCourier2013Oct-digitaledition.pdf
URL: https://cds.cern.ch/record/1603700/files/CERNCourier2013Oct-digitaledition.pdf
Error downloading CERNCourier2013Oct-digitaledition.pdf: 404 Client Error: Not Found for url: http://cds.cern.ch/record/1735007/files/CERNCourier2013Oct-digitaledition.pdf

Processing page 6...
Found 5 new article links on page 6

Found PDF: CERN Courier June 2013.pdf
URL: http://cds.cern.ch/record/1550751/files/CERN%20Courier%20June%202013.pdf
Error downloading CERN Courier June 2013.pdf: 404 Client Error: Not Found for url: http://cds.cern.ch/record/1734960/files/CERN%20Courier%20June%202013.pdf

Found PDF: CERN Courier digital edition May 2013.pdf
URL: http://cds.cern.ch/record/1544352/files/CERN%20Courier%20digital%20edition%20May%202013.pdf
Error downloading CERN Courier digital edition May 2013.pdf: 404 Client Error: Not Found for url: http://cds.cern.ch/record/1734947/files/CERN%20Courier%

# 2: Encode the CERN PDF documents

In [2]:
%%time
import os
from pathlib import Path
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb
from tqdm import tqdm
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

class PDFProcessor:
    def __init__(self, pdf_dir="cern_pdfs", db_dir="cern_vectordb"):
        # Check for API key
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY not found in .env file")
            
        self.pdf_dir = Path(pdf_dir)
        self.db_dir = Path(db_dir)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
            add_start_index=True,
        )
        
        # Initialize embeddings
        self.embeddings = OpenAIEmbeddings()
        
    def extract_text_from_pdf(self, pdf_path):
        """Extract text from a PDF file"""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                
                # Extract text from each page
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                    
                return text
        except Exception as e:
            print(f"Error extracting text from {pdf_path}: {e}")
            return None

    def process_pdfs(self):
        """Process all PDFs in the directory and return chunks with metadata"""
        all_chunks = []
        
        # Process each PDF file
        pdf_files = list(self.pdf_dir.glob("*.pdf"))
        for pdf_path in tqdm(pdf_files, desc="Processing PDFs"):
            text = self.extract_text_from_pdf(pdf_path)
            if text:
                # Split text into chunks
                chunks = self.text_splitter.create_documents(
                    texts=[text],
                    metadatas=[{"source": pdf_path.name}]
                )
                all_chunks.extend(chunks)
        
        return all_chunks

    def create_vector_db(self):
        """Create and populate the vector database"""
        # Get text chunks
        chunks = self.process_pdfs()
        
        if not chunks:
            print("No text chunks were created. Check the PDF processing.")
            return None
        
        print(f"\nCreating vector database with {len(chunks)} chunks...")
        
        # Create and persist the vector store
        vectordb = Chroma.from_documents(
            documents=chunks,
            embedding=self.embeddings,
            persist_directory=str(self.db_dir)
        )
        
        # Persist the database
        vectordb.persist()
        print(f"Vector database created and saved to {self.db_dir}")
        
        return vectordb

if __name__ == "__main__":
    try:
        processor = PDFProcessor()
        vectordb = processor.create_vector_db()
    except ValueError as e:
        print(f"Error: {e}")

Processing PDFs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [05:27<00:00,  5.74s/it]



Creating vector database with 17732 chunks...
Vector database created and saved to cern_vectordb
CPU times: user 6min 1s, sys: 4.03 s, total: 6min 5s
Wall time: 8min 43s


<timed exec>:85: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.


# 3: Chat with Cern Magazine

In [3]:
%%time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

class CERNResearchAssistant:
    def __init__(self, db_dir="cern_vectordb"):
        # Check for API key
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY not found in .env file")
            
        # Initialize the vector store
        self.vectorstore = Chroma(
            persist_directory=db_dir,
            embedding_function=OpenAIEmbeddings()
        )
        
        # Initialize the language model
        self.llm = ChatOpenAI(
            model="gpt-4-turbo-preview",
            temperature=0
        )
        
        # Create the retriever
        self.retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
        )
        
        # Setup the prompt template
        template = """You are a helpful research assistant with access to CERN Courier articles.
        Use the following articles to answer the question. If you can't answer the question based
        on the articles, say so clearly.

        Context articles:
        {context}

        Question: {question}

        Please provide a detailed answer with specific references to the articles when possible:"""
        
        self.prompt = ChatPromptTemplate.from_template(template)
        
        # Setup the RAG chain
        self.chain = (
            RunnableParallel(
                {"context": self.retriever, "question": RunnablePassthrough()}
            )
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
    
    def query(self, question):
        """Ask a question about CERN research"""
        try:
            response = self.chain.invoke(question)
            return response
        except Exception as e:
            return f"Error processing query: {e}"

def main():
    try:
        # Initialize the assistant
        assistant = CERNResearchAssistant()
        
        print("CERN Research Assistant Ready!")
        print("Ask questions about CERN research (type 'quit' to exit)")
        
        while True:
            question = input("\nYour question: ")
            if question.lower() in ['quit', 'exit', 'q']:
                break
                
            response = assistant.query(question)
            print("\nAssistant:", response)
            
    except ValueError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

<timed exec>:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.


CERN Research Assistant Ready!
Ask questions about CERN research (type 'quit' to exit)



Your question:  Neutrinos out of the woods



Assistant: The question "Neutrinos out of the woods" seems to metaphorically ask about the current state of neutrino research and whether significant progress or clarifications have been made in understanding neutrinos, moving beyond previous uncertainties or "woods." Based on the provided CERN Courier articles, we can piece together a narrative that shows significant advancements in neutrino physics, though challenges and mysteries remain.

1. **Understanding Neutrino Puzzles and Opportunities**:
   The November 2016 article highlights that neutrinos are at the center of many puzzles in particle physics but also present numerous opportunities for discovery. It mentions that a minimal extension of the Standard Model, which includes three right-handed neutrinos, could explain current experiments and observations, including dark matter. This suggests that while neutrinos are complex and challenging to study, they are integral to our understanding of the universe and potential new physic


Your question:  Grappling with dark energy



Assistant: The articles from CERN Courier provide a comprehensive overview of the current state of research and understanding regarding dark energy, its implications for our understanding of gravity, and the ongoing efforts to unravel its mysteries. Dark energy, alongside dark matter, represents one of the most profound enigmas in modern physics, accounting for about 95% of the energy in the universe and driving the accelerated expansion of the universe (CERN Courier Volume 57 Issue 4).

The observation that the universe is experiencing accelerated expansion is a key piece of evidence pointing towards the existence of dark energy. This phenomenon suggests that the universe contains an enormous amount of additional energy, which is not accounted for by visible matter alone (CERN Courier Volume 57 Issue 4). The existence of dark energy, along with dark matter, challenges our current understanding of gravity and the overall dynamics of the universe. Despite the overwhelming evidence for 


Your question:  exit


CPU times: user 1.73 s, sys: 523 ms, total: 2.25 s
Wall time: 4min 25s


# 4: Fine Tuning with OpenAI 

In [2]:
%%time

import os
import json
from pathlib import Path
import PyPDF2
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken
import time

# Load environment variables
load_dotenv()

class FineTunePrep:
    def __init__(self, pdf_dir="cern_pdfs", output_dir="finetune_data"):
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY not found in .env file")
            
        self.client = OpenAI()
        self.pdf_dir = Path(pdf_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        
        # Constants for token limits
        self.MAX_TOKENS_PER_EXAMPLE = 3000  # Leave room for system and user messages
        self.MIN_TOKENS_PER_EXAMPLE = 500   # Ensure meaningful content
        
    def count_tokens(self, text):
        """Count tokens in a text string"""
        return len(self.tokenizer.encode(text))

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from a PDF file"""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text
        except Exception as e:
            print(f"Error extracting text from {pdf_path}: {e}")
            return None

    def split_into_chunks(self, text):
        """Split text into chunks of appropriate token length"""
        chunks = []
        current_chunk = ""
        current_tokens = 0
        
        # Split into sentences (roughly)
        sentences = [s.strip() + "." for s in text.replace("\n", " ").split(".") if s.strip()]
        
        for sentence in sentences:
            sentence_tokens = self.count_tokens(sentence)
            
            # If single sentence is too long, split it into smaller parts
            if sentence_tokens > self.MAX_TOKENS_PER_EXAMPLE:
                words = sentence.split()
                temp_chunk = ""
                temp_tokens = 0
                
                for word in words:
                    word_tokens = self.count_tokens(word + " ")
                    if temp_tokens + word_tokens > self.MAX_TOKENS_PER_EXAMPLE:
                        if temp_tokens >= self.MIN_TOKENS_PER_EXAMPLE:
                            chunks.append(temp_chunk.strip())
                        temp_chunk = word + " "
                        temp_tokens = word_tokens
                    else:
                        temp_chunk += word + " "
                        temp_tokens += word_tokens
                
                if temp_tokens >= self.MIN_TOKENS_PER_EXAMPLE:
                    chunks.append(temp_chunk.strip())
                continue
            
            # If adding this sentence would exceed limit, save current chunk and start new one
            if current_tokens + sentence_tokens > self.MAX_TOKENS_PER_EXAMPLE:
                if current_tokens >= self.MIN_TOKENS_PER_EXAMPLE:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
                current_tokens = sentence_tokens
            else:
                current_chunk += sentence + " "
                current_tokens += sentence_tokens
        
        # Add the last chunk if it's long enough
        if current_tokens >= self.MIN_TOKENS_PER_EXAMPLE:
            chunks.append(current_chunk.strip())
        
        return chunks

    def create_training_examples(self, chunks):
        """Create training examples from text chunks"""
        examples = []
        
        for chunk in chunks:
            # Create the messages for this chunk
            messages = [
                {
                    "role": "system",
                    "content": "You are an expert on CERN and particle physics, trained to provide accurate information from CERN publications."
                },
                {
                    "role": "user",
                    "content": "What are the key findings or developments described in this CERN research?"
                },
                {
                    "role": "assistant",
                    "content": f"Based on the CERN publications: {chunk}"
                }
            ]
            
            # Verify total tokens
            total_tokens = sum(self.count_tokens(msg["content"]) for msg in messages)
            if total_tokens <= 4096:  # GPT-4's context window
                examples.append({"messages": messages})
            
        return examples

    def prepare_training_data(self):
        """Process PDFs and prepare training data"""
        all_examples = []
        pdf_files = list(self.pdf_dir.glob("*.pdf"))
        
        for pdf_path in tqdm(pdf_files, desc="Processing PDFs"):
            text = self.extract_text_from_pdf(pdf_path)
            if text:
                # First split text into appropriate chunks
                chunks = self.split_into_chunks(text)
                print(f"\nCreated {len(chunks)} chunks from {pdf_path.name}")
                
                # Create examples from chunks
                examples = self.create_training_examples(chunks)
                all_examples.extend(examples)
        
        # Save training data
        training_file_path = self.output_dir / "training_data.jsonl"
        with open(training_file_path, 'w', encoding='utf-8') as f:
            for example in all_examples:
                f.write(json.dumps(example) + '\n')
        
        print(f"\nCreated {len(all_examples)} valid training examples")
        print(f"Training data saved to {training_file_path}")
        return training_file_path

    def submit_fine_tuning_job(self, training_file_path):
        """Submit fine-tuning job to OpenAI"""
        try:
            # Upload the training file
            with open(training_file_path, 'rb') as f:
                training_file = self.client.files.create(
                    file=f,
                    purpose='fine-tune'
                )
            print(f"Training file uploaded with ID: {training_file.id}")
            
            # Create fine-tuning job
            job = self.client.fine_tuning.jobs.create(
                training_file=training_file.id,
                model="gpt-4o-mini-2024-07-18",
                hyperparameters={
                    "n_epochs": 2,
                    "learning_rate_multiplier": 0.1
                }
            )
            
            print(f"Fine-tuning job created with ID: {job.id}")
            return job.id
            
        except Exception as e:
            print(f"Error submitting fine-tuning job: {e}")
            return None

    def monitor_fine_tuning_job(self, job_id):
        """Monitor the status of a fine-tuning job"""
        print("\nMonitoring fine-tuning job...")
        
        while True:
            try:
                job = self.client.fine_tuning.jobs.retrieve(job_id)
                print(f"\nStatus: {job.status}")
                
                # Safely print additional info if available
                if hasattr(job, 'trained_tokens') and job.trained_tokens is not None:
                    print(f"Trained tokens: {job.trained_tokens:,}")
                if hasattr(job, 'training_accuracy') and job.training_accuracy is not None:
                    print(f"Training accuracy: {job.training_accuracy:.4f}")
                
                if job.status == 'succeeded':
                    print(f"\nFine-tuning completed successfully!")
                    print(f"Fine-tuned model ID: {job.fine_tuned_model}")
                    return job
                elif job.status == 'failed':
                    print(f"\nFine-tuning failed: {getattr(job, 'error', 'Unknown error')}")
                    return job
                elif job.status == 'cancelled':
                    print("\nFine-tuning job was cancelled")
                    return job
                
                time.sleep(60)
                
            except Exception as e:
                print(f"Error checking job status: {e}")
                time.sleep(60)

def main():
    try:
        prep = FineTunePrep()
        
        print("Step 1: Preparing training data...")
        training_file_path = prep.prepare_training_data()
        
        print("\nStep 2: Submitting fine-tuning job...")
        job_id = prep.submit_fine_tuning_job(training_file_path)
        
        if job_id:
            final_job = prep.monitor_fine_tuning_job(job_id)
            
            if getattr(final_job, 'status', None) == 'succeeded':
                print("\nFine-tuning process completed successfully!")
                print(f"You can now use your fine-tuned model with ID: {final_job.fine_tuned_model}")
                
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()



Step 1: Preparing training data...


Processing PDFs:   2%|██▎                                                                                                                                 | 1/57 [00:05<04:56,  5.29s/it]


Created 20 chunks from CERNCourier2024MayJun-digitaledition.pdf


Processing PDFs:   4%|████▋                                                                                                                               | 2/57 [00:10<04:35,  5.02s/it]


Created 19 chunks from CERNCourier2022NovDec-digitaledition.pdf


...

Processing PDFs:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 56/57 [05:16<00:04,  4.83s/it]


Created 15 chunks from CERNCourier2018Nov-digitaledition.pdf


Processing PDFs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [05:21<00:00,  5.64s/it]


Created 20 chunks from CERNCourier2024JanFeb-digitaledition.pdf

Created 1104 valid training examples
Training data saved to finetune_data/training_data.jsonl

Step 2: Submitting fine-tuning job...


Training file uploaded with ID: file-57Mt4LPzbyj7jMS49CcMc8
Fine-tuning job created with ID: ftjob-8tjSyNrQ1YLXahS4y7m2V82M

Monitoring fine-tuning job...

Status: validating_files

Status: validating_files

Status: validating_files

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

Status: running

S

# 5: RAG vs Fine-Tuning

In [4]:
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from dotenv import load_dotenv
import os
import time

# Load environment variables
load_dotenv()

class ModelComparison:
    def __init__(self, fine_tuned_model_id, db_dir="cern_vectordb"):
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY not found in .env file")
            
        self.client = OpenAI()
        self.fine_tuned_model_id = fine_tuned_model_id
        
        # Initialize RAG components
        self.vectorstore = Chroma(
            persist_directory=db_dir,
            embedding_function=OpenAIEmbeddings()
        )
        
        self.llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0
        )
        
        self.retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
        )
        
        template = """You are a helpful research assistant with access to CERN Courier articles.
        Use the following articles to answer the question. If you can't answer the question based
        on the articles, say so clearly.

        Context articles:
        {context}

        Question: {question}

        Please provide a detailed answer with specific references to the articles when possible:"""
        
        self.prompt = ChatPromptTemplate.from_template(template)
        
        self.rag_chain = (
            RunnableParallel(
                {"context": self.retriever, "question": RunnablePassthrough()}
            )
            | self.prompt
            | self.llm
            | StrOutputParser()
        )

    def query_fine_tuned_model(self, question):
        """Query the fine-tuned model"""
        try:
            start_time = time.time()
            
            response = self.client.chat.completions.create(
                model=self.fine_tuned_model_id,
                messages=[
                    {"role": "system", "content": "You are an expert on CERN and particle physics, trained to provide accurate information from CERN publications."},
                    {"role": "user", "content": question}
                ],
                temperature=0
            )
            
            end_time = time.time()
            
            return {
                'response': response.choices[0].message.content,
                'time': end_time - start_time
            }
            
        except Exception as e:
            return {
                'response': f"Error querying fine-tuned model: {e}",
                'time': 0
            }

    def query_rag(self, question):
        """Query the RAG system"""
        try:
            start_time = time.time()
            
            response = self.rag_chain.invoke(question)
            
            end_time = time.time()
            
            return {
                'response': response,
                'time': end_time - start_time
            }
            
        except Exception as e:
            return {
                'response': f"Error querying RAG system: {e}",
                'time': 0
            }

    def compare_responses(self, question):
        """Compare responses from both approaches"""
        print("\nQuerying both models...")
        
        # Get responses
        ft_result = self.query_fine_tuned_model(question)
        rag_result = self.query_rag(question)
        
        # Print comparison
        print("\n" + "="*50)
        print("Question:", question)
        print("="*50)
        
        print("\nFine-tuned Model Response:")
        print("-"*30)
        print(ft_result['response'])
        print(f"Response time: {ft_result['time']:.2f} seconds")
        
        print("\nRAG System Response:")
        print("-"*30)
        print(rag_result['response'])
        print(f"Response time: {rag_result['time']:.2f} seconds")
        
        return {
            'fine_tuned': ft_result,
            'rag': rag_result
        }

def main():
    # Replace with your fine-tuned model ID
    FINE_TUNED_MODEL_ID = "ft:gpt-4o-mini-2024-07-18:personal::AbZrBIYn"
    
    try:
        comparison = ModelComparison(FINE_TUNED_MODEL_ID)
        
        print("CERN Research Assistant Comparison")
        print("Compare Fine-tuned model vs RAG approach")
        print("Type 'quit' to exit")
        
        while True:
            question = input("\nYour question: ")
            if question.lower() in ['quit', 'exit', 'q']:
                break
                
            comparison.compare_responses(question)
            
    except ValueError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

CERN Research Assistant Comparison
Compare Fine-tuned model vs RAG approach
Type 'quit' to exit



Your question:  Grappling with dark energy



Querying both models...

Question: Grappling with dark energy

Fine-tuned Model Response:
------------------------------
The nature of dark energy is one of the most profound mysteries in modern physics. The term refers to the unknown cause of the observed accelerated expansion of the universe, which was discovered in 1998 by two independent teams of astronomers studying distant supernovae. The discovery was unexpected, as the prevailing view at the time was that the expansion of the universe was slowing down due to the gravitational attraction of matter. Instead, the observations revealed that the expansion is accelerating, suggesting that some form of energy is driving the acceleration. 

The simplest explanation for dark energy is the cosmological constant, a term that Albert Einstein introduced in 1917 to allow for a static universe. When it was discovered that the universe is expanding, Einstein abandoned the idea, calling it his “greatest blunder”. However, the cosmological cons


Your question:  exit


# 6: Fine Tunging on nvidia/Llama3-ChatQA-1.5-8B

In [2]:
import os
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import Dataset
from pathlib import Path
import PyPDF2
from tqdm import tqdm
from accelerate import Accelerator
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from huggingface_hub import login
from dotenv import load_dotenv
import gc
from pynvml import (
    nvmlInit,
    nvmlDeviceGetCount,
    nvmlDeviceGetHandleByIndex,
    nvmlDeviceGetMemoryInfo,
    NVMLError
)

def clean_gpu_memory():
    """
    Thoroughly clean GPU memory and print memory usage statistics
    """
    def get_gpu_memory_usage():
        try:
            nvmlInit()
            deviceCount = nvmlDeviceGetCount()
            memory_usage = []
            
            for i in range(deviceCount):
                handle = nvmlDeviceGetHandleByIndex(i)
                info = nvmlDeviceGetMemoryInfo(handle)
                memory_usage.append({
                    'device': i,
                    'used_mb': info.used / 1024**2,
                    'total_mb': info.total / 1024**2,
                    'used_percent': (info.used / info.total) * 100
                })
            
            return memory_usage
        except NVMLError as e:
            print(f"NVML Error: {e}")
            return None

    # Print initial memory usage
    memory_usage = get_gpu_memory_usage()
    if memory_usage:
        print("\nInitial GPU memory usage:")
        for gpu in memory_usage:
            print(f"GPU {gpu['device']}: {gpu['used_mb']:.2f}MB / {gpu['total_mb']:.2f}MB ({gpu['used_percent']:.2f}%)")

    # Empty CUDA cache
    torch.cuda.empty_cache()
    
    # Delete all variables in CUDA memory
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    del obj
        except Exception:
            pass
    
    # Run garbage collector
    gc.collect()
    
    # Reset peak memory stats
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_max_memory_allocated()
        torch.cuda.synchronize()
    
    # Print final memory usage
    memory_usage = get_gpu_memory_usage()
    if memory_usage:
        print("\nFinal GPU memory usage after cleaning:")
        for gpu in memory_usage:
            print(f"GPU {gpu['device']}: {gpu['used_mb']:.2f}MB / {gpu['total_mb']:.2f}MB ({gpu['used_percent']:.2f}%)")

# Set environment variable for memory allocation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64,garbage_collection_threshold:0.8"
clean_gpu_memory()

load_dotenv()

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

class LlamaFineTuner:
    def __init__(
        self,
        model_name="nvidia/Llama3-ChatQA-1.5-8B",
        cache_dir=None,
        pdf_dir="cern_pdfs",
        output_dir="llama_finetuned",
        device="cuda"
    ):
        self.pdf_dir = Path(pdf_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        self.logging_dir = self.output_dir / "logs"
        self.logging_dir.mkdir(exist_ok=True)
        self.device = device
        self.cache_dir = cache_dir
        
        try:
            print("Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                cache_dir=self.cache_dir,
                trust_remote_code=True,
                use_fast=True
            )
            
            if self.tokenizer.pad_token is None:
                self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            
            print("Loading model with optimized quantization...")
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                load_in_8bit_fp32_cpu_offload=True
            )
            
            # Calculate available GPU memory
            clean_gpu_memory()  # Clean memory before loading model
            gpu_memory = int(torch.cuda.get_device_properties(0).total_memory/1e9) - 2
            
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=quantization_config,
                device_map="auto",
                torch_dtype=torch.bfloat16,
                max_memory={0: f"{int(gpu_memory*0.7)}GB", "cpu": "32GB"},
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )
            
            # Enable gradient checkpointing
            self.model.gradient_checkpointing_enable()
            self.model.enable_input_require_grads()
            self.model.config.use_cache = False
            
            print("Initial model parameters:")
            print_trainable_parameters(self.model)
            
            print("Preparing model for QLoRA training...")
            self.model = prepare_model_for_kbit_training(self.model)
            
            # Configure LoRA
            lora_config = LoraConfig(
                r=8,
                lora_alpha=32,
                target_modules=["q_proj", "v_proj"],
                lora_dropout=0.1,
                bias="none",
                task_type=TaskType.CAUSAL_LM
            )
            
            print("Applying LoRA...")
            self.model = get_peft_model(self.model, lora_config)
            print_trainable_parameters(self.model)
            
            # Initialize accelerator
            self.accelerator = Accelerator(
                gradient_accumulation_steps=32,
                mixed_precision="bf16",
                project_dir=str(self.logging_dir),
                split_batches=True,
                dispatch_batches=True
            )
            
        except Exception as e:
            raise RuntimeError(f"Error initializing model and tokenizer: {str(e)}")

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from a PDF file."""
        try:
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                return text
        except Exception as e:
            print(f"Error extracting text from {pdf_path}: {e}")
            return None

    def prepare_training_data(self, max_length=512):
        """Prepare training data from PDF files."""
        if not self.pdf_dir.exists():
            raise ValueError(f"PDF directory not found: {self.pdf_dir}")
        
        training_data = []
        pdf_files = list(self.pdf_dir.glob("*.pdf"))
        
        if not pdf_files:
            raise ValueError(f"No PDF files found in {self.pdf_dir}")
        
        for pdf_path in tqdm(pdf_files, desc="Processing PDFs"):
            text = self.extract_text_from_pdf(pdf_path)
            if text:
                entry = {
                    "instruction": "Analyze and explain the following research:",
                    "input": text,
                    "output": "This research discusses: " + text[:500]
                }
                training_data.append(entry)
        
        if not training_data:
            raise ValueError("No valid training data could be extracted from PDFs")
        
        dataset = Dataset.from_list(training_data)
        
        def tokenize_function(examples):
            prompts = []
            for inst, inp in zip(examples["instruction"], examples["input"]):
                prompt = f"[INST] {inst}\n{inp} [/INST]"
                prompts.append(prompt)
            
            return self.tokenizer(
                prompts,
                truncation=True,
                max_length=max_length,
                padding="max_length",
                return_tensors="pt"
            )
        
        return dataset.map(
            tokenize_function,
            batched=True,
            remove_columns=dataset.column_names
        )

    def train(self, num_epochs=3, batch_size=1):
        """Train the model."""
        try:
            print("Preparing training data...")
            train_dataset = self.prepare_training_data()
            
            training_args = TrainingArguments(
                output_dir=str(self.output_dir),
                num_train_epochs=num_epochs,
                per_device_train_batch_size=batch_size,
                gradient_accumulation_steps=32,
                warmup_steps=50,
                logging_steps=10,
                save_steps=100,
                learning_rate=2e-4,
                bf16=True,
                optim="paged_adamw_32bit",
                logging_dir=str(self.logging_dir),
                group_by_length=True,
                gradient_checkpointing=True,
                max_grad_norm=0.3,
                save_total_limit=2,
                evaluation_strategy="no",
                report_to="tensorboard",
                remove_unused_columns=False,
                lr_scheduler_type="cosine",
                weight_decay=0.01
            )
            
            trainer = Trainer(
                model=self.model,
                args=training_args,
                train_dataset=train_dataset,
                data_collator=DataCollatorForLanguageModeling(
                    tokenizer=self.tokenizer,
                    mlm=False
                ),
            )
            
            print("Starting training...")
            with torch.cuda.amp.autocast(dtype=torch.bfloat16):
                trainer.train()
            
            print("Saving model...")
            trainer.save_model(str(self.output_dir / "final_model"))
            self.tokenizer.save_pretrained(
                str(self.output_dir / "final_model"),
                safe_serialization=True
            )
            
            print("Training complete!")
            
        except Exception as e:
            raise RuntimeError(f"Error during training: {str(e)}")

def main():
    try:
        if not torch.cuda.is_available():
            raise RuntimeError("This script requires a CUDA-capable GPU")
        
        # Print GPU info
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")
        
        # Clean GPU memory before starting
        clean_gpu_memory()
        
        # Set memory optimizations
        torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = False
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        torch.cuda.set_per_process_memory_fraction(0.75)
        
        # Create cache directory
        cache_dir = Path.home() / ".cache" / "huggingface"
        cache_dir.mkdir(parents=True, exist_ok=True)
        
        # Initialize and train
        print("Initializing fine-tuner...")
        finetuner = LlamaFineTuner(cache_dir=str(cache_dir))
        finetuner.train()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()


Initial GPU memory usage:
GPU 0: 380.31MB / 24576.00MB (1.55%)


/home/lence/anaconda3/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:359: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/home/lence/anaconda3/lib/python3.11/site-packages/torch/cuda/memory.py:330: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(



Final GPU memory usage after cleaning:
GPU 0: 637.25MB / 24576.00MB (2.59%)
Using GPU: NVIDIA GeForce RTX 3090
Available GPU memory: 25.44 GB

Initial GPU memory usage:
GPU 0: 637.25MB / 24576.00MB (2.59%)

Final GPU memory usage after cleaning:
GPU 0: 637.25MB / 24576.00MB (2.59%)
Initializing fine-tuner...
Loading tokenizer...


Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading model with optimized quantization...

Initial GPU memory usage:
GPU 0: 637.25MB / 24576.00MB (2.59%)

Final GPU memory usage after cleaning:
GPU 0: 637.25MB / 24576.00MB (2.59%)


...

/home/lence/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training...


/home/lence/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


Saving model...
Training complete!


# 7: Chat with the Fine-Tuned nvidia/Llama3-ChatQA-1.5-8B

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel
from pathlib import Path

class ModelChat:
    def __init__(
        self,
        base_model_name="nvidia/Llama3-ChatQA-1.5-8B",
        finetuned_path="llama_finetuned/final_model",
        max_sequence_length=2048
    ):
        try:
            print("Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(
                base_model_name,
                trust_remote_code=True,
                use_fast=True
            )
            
            # Set padding parameters
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            self.max_sequence_length = max_sequence_length
            
            print("Loading base model...")
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
            
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                quantization_config=quantization_config,
                trust_remote_code=True
            )
            
            print("Loading fine-tuned adapters...")
            self.model = PeftModel.from_pretrained(
                base_model,
                finetuned_path,
                torch_dtype=torch.float16,
                device_map="auto"
            )
            
            self.model.eval()
            print("Model loaded successfully!")
            print(f"Model is on device: {self.model.device}")
            
        except Exception as e:
            raise RuntimeError(f"Error initializing model: {str(e)}")
    
    def generate_response(self, instruction, max_new_tokens=256, temperature=0.7):
        try:
            # Format the input
            prompt = f"[INST] {instruction} [/INST]"
            print(f"\nDebug - Prompt: {prompt}")
            
            # Tokenize input
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=self.max_sequence_length
            )
            print(f"Debug - Input shape: {inputs.input_ids.shape}")
            
            # Move to GPU
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            
            # Generate with minimal parameters first
            with torch.no_grad():
                outputs = self.model.generate(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    max_new_tokens=max_new_tokens,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id
                )
            
            print(f"Debug - Output shape: {outputs.shape}")
            
            # Decode response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"Debug - Raw response: {response}")
            
            # Clean response
            response = response.replace(prompt, "").strip()
            print(f"Debug - Cleaned response: {response}")
            
            return response
            
        except Exception as e:
            print(f"Debug - Error in generate_response: {str(e)}")
            return f"Error generating response: {str(e)}"
    
    def chat(self):
        print("\nStarting chat session with the fine-tuned model")
        print("Type 'quit' to exit, 'clear' to clear the conversation")
        print("-" * 50)
        
        while True:
            try:
                user_input = input("\nYou: ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'bye']:
                    print("\nGoodbye!")
                    break
                
                if not user_input:
                    continue
                
                print("\nGenerating response...")
                response = self.generate_response(user_input)
                
                if response:
                    print(f"\nModel: {response}")
                else:
                    print("\nModel: No response generated. Please try again.")
                
            except KeyboardInterrupt:
                print("\n\nInterrupted by user. Type 'quit' to exit or continue chatting.")
            except Exception as e:
                print(f"\nError during chat: {str(e)}")

def main():
    try:
        if not torch.cuda.is_available():
            raise RuntimeError("This script requires a CUDA-capable GPU")
        
        print("\nInitializing chat model...")
        chat_model = ModelChat()
        chat_model.chat()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


Initializing chat model...
Loading tokenizer...
Loading base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading fine-tuned adapters...
Model loaded successfully!
Model is on device: cuda:0

Starting chat session with the fine-tuned model
Type 'quit' to exit, 'clear' to clear the conversation
--------------------------------------------------



You:  Grappling with dark energy



Generating response...

Debug - Prompt: [INST] Grappling with dark energy [/INST]
Debug - Input shape: torch.Size([1, 12])


...


Generating response...

Debug - Prompt: [INST] what is Grappling with dark energy [/INST]
Debug - Input shape: torch.Size([1, 14])
Debug - Output shape: torch.Size([1, 15])
Debug - Raw response: [INST] what is Grappling with dark energy [/INST]
Debug - Cleaned response: 

Model: No response generated. Please try again.



You:  What is driving the accelerated expansion of the Universe?



Generating response...

Debug - Prompt: [INST] What is driving the accelerated expansion of the Universe? [/INST]
Debug - Input shape: torch.Size([1, 16])
Debug - Output shape: torch.Size([1, 58])
Debug - Raw response: [INST] What is driving the accelerated expansion of the Universe? [/INST] [INST] Can we test the nature of dark energy with future cosmology surveys? [/INST] [INST] Do we finally understand the origin of the large scale structure of the Universe? [/INST]
Debug - Cleaned response: [INST] Can we test the nature of dark energy with future cosmology surveys? [/INST] [INST] Do we finally understand the origin of the large scale structure of the Universe? [/INST]

Model: [INST] Can we test the nature of dark energy with future cosmology surveys? [/INST] [INST] Do we finally understand the origin of the large scale structure of the Universe? [/INST]



You:  exit



Goodbye!


# 8: Now compaire OpenAI RAG vs OpenAI Fine-Tune vs nvidia/Llama3-ChatQA-1.5-8B

In [2]:
import os
import time
import torch
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel
from dotenv import load_dotenv
from pathlib import Path

class ModelComparison:
    def __init__(
        self,
        fine_tuned_model_id="ft:gpt-4o-mini-2024-07-18:personal::AbZrBIYn",
        base_model_name="nvidia/Llama3-ChatQA-1.5-8B",
        finetuned_path="llama_finetuned/final_model",
        db_dir="cern_vectordb"
    ):
        load_dotenv()
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY not found in .env file")
        
        # Initialize OpenAI client
        self.client = OpenAI()
        self.fine_tuned_model_id = fine_tuned_model_id
        
        # Initialize RAG components
        print("Initializing RAG system...")
        self.vectorstore = Chroma(
            persist_directory=db_dir,
            embedding_function=OpenAIEmbeddings()
        )
        
        self.llm = ChatOpenAI(model="gpt-4", temperature=0)
        
        self.retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
        )
        
        # Setup RAG prompt template
        template = """You are a CERN research assistant. Use the following articles to answer the question.
        If you cannot answer based on the articles, say so clearly.

        Context articles:
        {context}

        Question: {question}

        Answer with specific references to the articles:"""
        
        self.prompt = ChatPromptTemplate.from_template(template)
        
        # Setup RAG chain
        self.rag_chain = (
            RunnableParallel(
                {"context": self.retriever, "question": RunnablePassthrough()}
            )
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
        
        # Initialize LLaMA model
        print("Loading LLaMA model...")
        try:
            # Initialize tokenizer
            self.llama_tokenizer = AutoTokenizer.from_pretrained(
                base_model_name,
                trust_remote_code=True,
                use_fast=True
            )
            
            if self.llama_tokenizer.pad_token is None:
                self.llama_tokenizer.pad_token = self.llama_tokenizer.eos_token
            
            # Setup quantization
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
            
            # Load base model
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                quantization_config=quantization_config,
                trust_remote_code=True
            )
            
            # Load LoRA adapters
            self.llama_model = PeftModel.from_pretrained(
                base_model,
                finetuned_path,
                torch_dtype=torch.float16,
                device_map="auto"
            )
            
            self.llama_model.eval()
            print("LLaMA model loaded successfully!")
            
        except Exception as e:
            print(f"Warning: Could not load LLaMA model: {e}")
            self.llama_model = None
            self.llama_tokenizer = None

    def query_llama(self, question, max_length=512):
        """Query the LoRA-fine-tuned LLaMA model"""
        if not self.llama_model or not self.llama_tokenizer:
            return {
                'response': "LLaMA model not available",
                'time': 0
            }
        
        try:
            start_time = time.time()
            
            # Prepare input
            prompt = f"[INST] {question} [/INST]"
            inputs = self.llama_tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=max_length,
                padding=True
            ).to(self.llama_model.device)
            
            # Generate response
            with torch.no_grad():
                outputs = self.llama_model.generate(
                    **inputs,
                    max_new_tokens=max_length,
                    temperature=0.7,
                    top_p=0.95,
                    repetition_penalty=1.1,
                    do_sample=True,
                    pad_token_id=self.llama_tokenizer.pad_token_id,
                    eos_token_id=self.llama_tokenizer.eos_token_id
                )
            
            # Decode response
            response = self.llama_tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response.replace(prompt, "").strip()
            
            end_time = time.time()
            
            return {
                'response': response,
                'time': end_time - start_time
            }
            
        except Exception as e:
            return {
                'response': f"Error querying LLaMA model: {e}",
                'time': 0
            }

    def query_fine_tuned_gpt(self, question):
        """Query the OpenAI fine-tuned model"""
        try:
            start_time = time.time()
            
            response = self.client.chat.completions.create(
                model=self.fine_tuned_model_id,
                messages=[
                    {"role": "system", "content": "You are an expert on CERN and particle physics research."},
                    {"role": "user", "content": question}
                ],
                temperature=0.7
            )
            
            end_time = time.time()
            
            return {
                'response': response.choices[0].message.content,
                'time': end_time - start_time
            }
            
        except Exception as e:
            return {
                'response': f"Error querying fine-tuned GPT: {e}",
                'time': 0
            }

    def query_rag(self, question):
        """Query the RAG system"""
        try:
            start_time = time.time()
            response = self.rag_chain.invoke(question)
            end_time = time.time()
            
            return {
                'response': response,
                'time': end_time - start_time
            }
            
        except Exception as e:
            return {
                'response': f"Error querying RAG system: {e}",
                'time': 0
            }

    def compare_responses(self, question):
        """Compare responses from all three models"""
        print("\nProcessing your question across all models...")
        
        # Get responses
        rag_result = self.query_rag(question)
        ft_result = self.query_fine_tuned_gpt(question)
        llama_result = self.query_llama(question)
        
        # Print results
        print("\n" + "="*80)
        print(f"Question: {question}")
        print("="*80)
        
        print("\n1. RAG System (GPT-4 + CERN Articles)")
        print("-"*50)
        print(rag_result['response'])
        print(f"Response time: {rag_result['time']:.2f} seconds")
        
        print("\n2. Fine-tuned GPT-4")
        print("-"*50)
        print(ft_result['response'])
        print(f"Response time: {ft_result['time']:.2f} seconds")
        
        print("\n3. Fine-tuned LLaMA (with LoRA)")
        print("-"*50)
        print(llama_result['response'])
        print(f"Response time: {llama_result['time']:.2f} seconds")
        
        return {
            'rag': rag_result,
            'fine_tuned_gpt': ft_result,
            'llama': llama_result
        }

def main():
    try:
        print("Initializing Model Comparison System...")
        comparison = ModelComparison()
        
        print("\nCERN Research Assistant - Model Comparison")
        print("Compare: RAG vs Fine-tuned GPT-4 vs Fine-tuned LLaMA")
        print("Type 'quit' to exit")
        
        while True:
            question = input("\nYour question: ").strip()
            if question.lower() in ['quit', 'exit', 'q']:
                break
            if not question:
                continue
                
            comparison.compare_responses(question)
            
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Initializing Model Comparison System...
Initializing RAG system...


/tmp/ipykernel_395023/2066110667.py:37: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectorstore = Chroma(


Loading LLaMA model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LLaMA model loaded successfully!

CERN Research Assistant - Model Comparison
Compare: RAG vs Fine-tuned GPT-4 vs Fine-tuned LLaMA
Type 'quit' to exit



Your question:  What is driving the accelerated expansion of the Universe?



Processing your question across all models...

Question: What is driving the accelerated expansion of the Universe?

1. RAG System (GPT-4 + CERN Articles)
--------------------------------------------------
The accelerated expansion of the Universe is broadly attributed to what is referred to as dark energy, as mentioned in the first document. However, the exact source and physics of dark energy remain unknown and are an ongoing area of research. One idea is that the cosmological constant, initially introduced by Einstein, is linked to the vacuum energy. However, this can't be the final answer because summing the contributions from the presumed quantum states in the universe produces a number for the expansion rate that is significantly higher than observed. The fourth document also mentions that the nature of this so-called dark energy is one of the most important puzzles in modern fundamental physics. It could be due to a cosmological constant, a new dynamical field, a deviation from


Your question:  exit


# 9: Build a LLM from scratch on the CERN PDFs

## Cell 1: Config and Base Classes

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import numpy as np
import PyPDF2
from pathlib import Path
from tqdm import tqdm
import math
import logging
from typing import Optional, Tuple
import wandb
import gc
import os

class TransformerConfig:
    def __init__(
        self,
        vocab_size: int = 32000,
        max_sequence_length: int = 512,
        d_model: int = 768,
        n_heads: int = 12,
        n_layers: int = 6,
        d_ff: int = 3072,
        dropout: float = 0.1,
        pad_token_id: int = 0
    ):
        self.vocab_size = vocab_size
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.d_ff = d_ff
        self.dropout = dropout
        self.pad_token_id = pad_token_id

class MultiHeadAttention(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.d_model = config.d_model
        self.n_heads = config.n_heads
        assert self.d_model % self.n_heads == 0
        
        self.d_k = self.d_model // self.n_heads
        self.w_q = nn.Linear(config.d_model, config.d_model)
        self.w_k = nn.Linear(config.d_model, config.d_model)
        self.w_v = nn.Linear(config.d_model, config.d_model)
        self.w_o = nn.Linear(config.d_model, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        
    def attention(self, q, k, v, mask=None):
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        return torch.matmul(attn, v), attn
    
    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)
        
        q = self.w_q(q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        k = self.w_k(k).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        v = self.w_v(v).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        
        x, attn = self.attention(q, k, v, mask)
        
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        return self.w_o(x)

## Cell 2: Model Architecture

In [9]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.fc1 = nn.Linear(config.d_model, config.d_ff)
        self.fc2 = nn.Linear(config.d_ff, config.d_model)
        self.dropout = nn.Dropout(config.dropout)
        
    def forward(self, x):
        return self.fc2(self.dropout(F.gelu(self.fc1(x))))

class TransformerBlock(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.attention = MultiHeadAttention(config)
        self.norm1 = nn.LayerNorm(config.d_model)
        self.norm2 = nn.LayerNorm(config.d_model)
        self.feed_forward = PositionwiseFeedForward(config)
        self.dropout = nn.Dropout(config.dropout)
        
    def forward(self, x, mask=None):
        attn_output = self.attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()
        pe = torch.zeros(config.max_sequence_length, config.d_model)
        position = torch.arange(0, config.max_sequence_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, config.d_model, 2).float() * 
                           (-math.log(10000.0) / config.d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(config.dropout)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class CustomLLM(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.config = config
        
        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.positional_encoding = PositionalEncoding(config)
        
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(config) for _ in range(config.n_layers)
        ])
        
        self.final_layer_norm = nn.LayerNorm(config.d_model)
        self.output_projection = nn.Linear(config.d_model, config.vocab_size, bias=False)
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.ones_(module.weight)
            torch.nn.init.zeros_(module.bias)
            
    def forward(
        self, 
        input_ids: torch.LongTensor,
        attention_mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        x = self.token_embedding(input_ids)
        x = self.positional_encoding(x)
        
        if attention_mask is not None:
            attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
            
        for block in self.transformer_blocks:
            x = block(x, attention_mask)
            
        x = self.final_layer_norm(x)
        logits = self.output_projection(x)
        
        return logits

## Cell 3: Dataset and Training Classes

In [10]:
class CERNDataset(Dataset):
    def __init__(
        self,
        pdf_dir: str,
        tokenizer_name: str = "gpt2",
        max_length: int = 512,
        min_text_length: int = 100
    ):
        self.pdf_dir = Path(pdf_dir)
        if not self.pdf_dir.exists():
            raise ValueError(f"PDF directory {pdf_dir} does not exist")
            
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
                logging.info("Set pad_token to eos_token")
        except Exception as e:
            raise RuntimeError(f"Failed to load tokenizer: {str(e)}")
            
        self.max_length = max_length
        self.min_text_length = min_text_length
        
        # Process PDFs
        self.texts = []
        self._process_pdfs()
        
        if not self.texts:
            raise ValueError(f"No valid texts found in {pdf_dir}")
            
        # Process texts in batches
        self._tokenize_texts()
        
    def _tokenize_texts(self):
        self.input_ids = []
        self.attention_masks = []
        
        batch_size = 32
        for i in tqdm(range(0, len(self.texts), batch_size), desc="Tokenizing texts"):
            batch_texts = self.texts[i:i + batch_size]
            try:
                encodings = self.tokenizer(
                    batch_texts,
                    truncation=True,
                    padding="max_length",
                    max_length=self.max_length,
                    return_tensors="pt"
                )
                self.input_ids.extend(encodings['input_ids'])
                self.attention_masks.extend(encodings['attention_mask'])
            except Exception as e:
                logging.error(f"Error tokenizing batch {i}: {str(e)}")
                continue
        
        if not self.input_ids:
            raise ValueError("No texts were successfully tokenized")
            
        try:
            self.input_ids = torch.stack([t for t in self.input_ids if isinstance(t, torch.Tensor)])
            self.attention_masks = torch.stack([t for t in self.attention_masks if isinstance(t, torch.Tensor)])
            logging.info(f"Successfully processed {len(self.input_ids)} sequences")
        except Exception as e:
            raise RuntimeError(f"Failed to stack tensors: {str(e)}")
        
    def _process_pdfs(self):
        pdf_files = list(self.pdf_dir.glob("*.pdf"))
        if not pdf_files:
            raise ValueError(f"No PDF files found in {self.pdf_dir}")
            
        for pdf_path in tqdm(pdf_files, desc="Processing PDFs"):
            try:
                with open(pdf_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() + "\n"
                    
                    words = text.split()
                    chunk_size = self.max_length * 4
                    
                    for i in range(0, len(words), chunk_size):
                        chunk = " ".join(words[i:i + chunk_size])
                        if len(chunk.strip()) >= self.min_text_length:
                            self.texts.append(chunk)
                        
            except Exception as e:
                logging.warning(f"Error processing {pdf_path}: {e}")
                continue
                
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        item = {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': torch.roll(self.input_ids[idx], -1)
        }
        item['labels'][-1] = self.tokenizer.pad_token_id
        return item

class Trainer:
    def __init__(
        self,
        model: CustomLLM,
        train_dataset: CERNDataset,
        val_dataset: Optional[CERNDataset] = None,
        batch_size: int = 8,
        learning_rate: float = 3e-4,
        num_epochs: int = 10,
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
        wandb_project: Optional[str] = None,
        checkpoint_dir: str = "checkpoints"
    ):
        self.model = model.to(device)
        self.device = device
        self.num_epochs = num_epochs
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(exist_ok=True)
        
        try:
            self.train_loader = DataLoader(
                train_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=4,
                pin_memory=True
            )
            
            self.val_loader = None
            if val_dataset:
                self.val_loader = DataLoader(
                    val_dataset,
                    batch_size=batch_size,
                    shuffle=False,
                    num_workers=4,
                    pin_memory=True
                )
        except Exception as e:
            raise RuntimeError(f"Failed to create data loaders: {str(e)}")
        
        self.optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=learning_rate,
            betas=(0.9, 0.95),
            weight_decay=0.1
        )
        
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer,
            T_max=len(self.train_loader) * num_epochs
        )
        
        if wandb_project:
            try:
                wandb.init(project=wandb_project)
            except Exception as e:
                logging.error(f"Failed to initialize wandb: {str(e)}")
                
    def save_checkpoint(self, epoch: int, loss: float, is_best: bool = False):
        try:
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scheduler_state_dict': self.scheduler.state_dict(),
                'loss': loss
            }
            
            filename = self.checkpoint_dir / f'checkpoint_epoch_{epoch}.pt'
            torch.save(checkpoint, filename)
            
            if is_best:
                best_filename = self.checkpoint_dir / 'best_model.pt'
                torch.save(checkpoint, best_filename)
        except Exception as e:
            logging.error(f"Failed to save checkpoint: {str(e)}")
            
    def train(self):
        best_val_loss = float('inf')
        
        try:
            for epoch in range(self.num_epochs):
                self.model.train()
                total_loss = 0
                train_pbar = tqdm(self.train_loader, desc=f"Training Epoch {epoch+1}")
                
                for batch in train_pbar:
                    try:
                        input_ids = batch['input_ids'].to(self.device)
                        attention_mask = batch['attention_mask'].to(self.device)
                        labels = batch['labels'].to(self.device)
                        
                        self.optimizer.zero_grad()
                        outputs = self.model(input_ids, attention_mask)
                        
                        loss = F.cross_entropy(
                            outputs.view(-1, outputs.size(-1)),
                            labels.view(-1),
                            ignore_index=self.model.config.pad_token_id
                        )
                        
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                        self.optimizer.step()
                        self.scheduler.step()
                        
                        total_loss += loss.item()
                        train_pbar.set_postfix({'loss': loss.item()})
                        
                        if wandb.run:
                            wandb.log({
                                'train_batch_loss': loss.item(),
                                'learning_rate': self.scheduler.get_last_lr()[0]
                            })
                    except Exception as e:
                        logging.error(f"Error in training batch: {str(e)}")
                        continue
                
                avg_train_loss = total_loss / len(self.train_loader)
                logging.info(f"Epoch {epoch+1} average training loss: {avg_train_loss:.4f}")
                
                if self.val_loader:
                    val_loss = self.evaluate()
                    logging.info(f"Epoch {epoch+1} validation loss: {val_loss:.4f}")
                    
                    is_best = val_loss < best_val_loss
                    if is_best:
                        best_val_loss = val_loss
                    
                    self.save_checkpoint(epoch + 1, val_loss, is_best)
                
                gc.collect()
                torch.cuda.empty_cache()
                
        except Exception as e:
            logging.error(f"Training failed: {str(e)}")
            raise
    
    def evaluate(self):
        self.model.eval()
        total_loss = 0
        n_batches = len(self.val_loader)
        
        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc="Evaluating"):
                try:
                    input_ids = batch['input_ids'].to(self.device)
                    attention_mask = batch['attention_mask'].to(self.device)
                    labels = batch['labels'].to(self.device)
                    
                    outputs = self.model(input_ids, attention_mask)
                    loss = F.cross_entropy(
                        outputs.view(-1, outputs.size(-1)),
                        labels.view(-1),
                        ignore_index=self.model.config.pad_token_id
                    )
                    
                    total_loss += loss.item()
                except Exception as e:
                    logging.error(f"Error in validation batch: {str(e)}")
                    n_batches -= 1
                    continue
        
        if n_batches == 0:
            raise RuntimeError("No valid batches in validation")
            
        return total_loss / n_batches

## Cell 4: Main Function

In [13]:
import os
import shutil
from pathlib import Path
import random

def split_pdfs(source_dir='cern_pdfs', train_ratio=0.8):
    source_path = Path(source_dir)
    train_path = source_path / 'train'
    val_path = source_path / 'val'
    
    # Create directories
    train_path.mkdir(exist_ok=True)
    val_path.mkdir(exist_ok=True)
    
    # Get all PDFs
    pdf_files = list(source_path.glob('*.pdf'))
    random.shuffle(pdf_files)
    
    # Calculate split
    split_idx = int(len(pdf_files) * train_ratio)
    train_files = pdf_files[:split_idx]
    val_files = pdf_files[split_idx:]
    
    # Move files
    for pdf in train_files:
        shutil.move(str(pdf), str(train_path / pdf.name))
    
    for pdf in val_files:
        shutil.move(str(pdf), str(val_path / pdf.name))
    
    print(f"Moved {len(train_files)} files to train/")
    print(f"Moved {len(val_files)} files to val/")

split_pdfs()

def main():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('training.log'),
            logging.StreamHandler()
        ]
    )
    
    try:
        config = TransformerConfig(
            vocab_size=50257,  # GPT-2 vocabulary size
            max_sequence_length=512,
            d_model=768,
            n_heads=12,
            n_layers=6,
            d_ff=3072,
            dropout=0.1
        )
        
        os.makedirs("cern_pdfs/train", exist_ok=True)
        os.makedirs("cern_pdfs/val", exist_ok=True)
        os.makedirs("checkpoints", exist_ok=True)
        
        logging.info("Creating datasets...")
        train_dataset = CERNDataset(
            "cern_pdfs/train",
            tokenizer_name="gpt2",
            max_length=config.max_sequence_length
        )
        val_dataset = CERNDataset(
            "cern_pdfs/val", 
            tokenizer_name="gpt2",
            max_length=config.max_sequence_length
        )
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = CustomLLM(config)
        
        trainer = Trainer(
            model=model,
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            batch_size=8,
            learning_rate=3e-4,
            num_epochs=10,
            device=device,
            wandb_project="cern-llm",
            checkpoint_dir="checkpoints"
        )
        
        trainer.train()
        
    except Exception as e:
        logging.error(f"Training failed: {str(e)}", exc_info=True)
        raise
    finally:
        if wandb.run:
            wandb.finish()
        gc.collect()
        torch.cuda.empty_cache()

if __name__ == "__main__":
    main()

2024-12-11 08:44:30,910 - INFO - Creating datasets...


Moved 0 files to train/
Moved 0 files to val/


/home/lence/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-12-11 08:44:31,189 - INFO - Set pad_token to eos_token
Tokenizing texts: 100%|███████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 32.45it/s]
2024-12-11 08:48:54,170 - INFO - Successfully processed 841 sequences
2024-12-11 08:48:54,401 - INFO - Set pad_token to eos_token
Tokenizing texts: 100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 36.63it/s]
2024-12-11 08:50:01,870 - INFO - Successfully processed 226 sequences
wandb: Currently logged in as: ikdataml (ikdataml-lence). Use `wandb 

Training Epoch 1:   0%|                                                                                       | 0/106 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` bef

learning_rate,████████▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
train_batch_loss,▇▅▂▅▄▇▄▄▅▇▅▆▅▆▇▃▄▄▃▇▂▃▆▅▇▂▄▅▄▂▂▅▅▆▄▃▁█▅▆
learning_rate,0
train_batch_loss,7.83195
